### LSTM

1. Why LSTM could work here
- LSTMs are neural networks designed for time-series, capable of remembering past states (lags) automatically.

- Can capture nonlinear patterns, spikes, and delayed effects better than Prophet.

- Handles multi-step forecasting by learning sequential dependencies.

2. Basic Setup
We’ll:

1. Use one commodity at a time (e.g., Beans Wholesale first).

2. Scale prices (LSTMs are sensitive to scale).

3. Create sequences (lags):

 - Input: last n days (e.g., 30)

 - Output: next day’s price.

4. Train an LSTM model using Keras/TensorFlow.

5. Evaluate on last 60 days using RMSE, MAE, MAPE, R².

In [57]:
# we select single series (Beans Wholesale)
commodity = "Beans"
price_col = "WholesaleUnitPrice"
df_series = df_p[df_p["Commodity"] == commodity][["Date", price_col]].dropna().sort_values("Date")

# Scale prices (LSTM needs scaled inputs)
scaler = MinMaxScaler()
prices_scaled = scaler.fit_transform(df_series[[price_col]])


In [58]:
# Create sequences (e.g., 30 days input → 1 day output)
def create_sequences(data, window=30):
    X, y = [], []
    for i in range(window, len(data)):
        X.append(data[i-window:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

window_size = 30
X, y = create_sequences(prices_scaled, window_size)



SPLIT TRAIN/TEST DATA FOR THE LAST 60 DAYS AND RESHAPE FOR LSTM SAMPLES,TIMESTEPS AND ALSO FEATURES

In [59]:
# Split train/test (last 60 days as test)
split = len(X) - 60
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Reshape for LSTM (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

### Build LSTM model


In [60]:
model = Sequential()
model.add(LSTM(64, return_sequences=False, input_shape=(window_size, 1)))
model.add(Dropout(0.2))
model.add(Dense(1))  # Predict next price

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=30, batch_size=16, verbose=1)


c:\Users\User\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 19s 11ms/step - loss: 0.0307
Epoch 2/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 0.0219
Epoch 3/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - loss: 0.0217
Epoch 4/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 0.0214
Epoch 5/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 12s 11ms/step - loss: 0.0200
Epoch 6/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - loss: 0.0202
Epoch 7/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 12s 11ms/step - loss: 0.0202
Epoch 8/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - loss: 0.0207
Epoch 9/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 12s 11ms/step - loss: 0.0209
Epoch 10/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 12s 11ms/step - loss: 0.0204
Epoch 11/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 12s 11ms/step - loss: 0.0204
Epoch 12/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 12s 11ms/step - loss: 0.0200
Epoch 13/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 12s 11ms/step - loss: 0.0200
Epoch 14/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - loss: 0.0200
E

### Prediction and evalutuation of the model performance

In [61]:
# Predict
pred_scaled = model.predict(X_test)
pred_prices = scaler.inverse_transform(pred_scaled.reshape(-1, 1))
actual_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

# Evaluate
mae = mean_absolute_error(actual_prices, pred_prices)
rmse = mean_squared_error(actual_prices, pred_prices, squared=False)
r2 = r2_score(actual_prices, pred_prices)
mape = np.mean(np.abs((actual_prices - pred_prices) / actual_prices)) * 100

print(f"LSTM Results for {commodity} ({price_col}):")
print(f"MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.4f}, MAPE: {mape:.2f}%")


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 316ms/step
LSTM Results for Beans (WholesaleUnitPrice):
MAE: 23.20, RMSE: 27.96, R²: 0.0028, MAPE: 19.45%


THE INTERPRETATION OF THE MODEL:
 


### HYPERTUNE THE MODEL DUE TO THE POOR PERFORMANCE

In [62]:
### Data Preparation and rollng of features

In [63]:
commodity = "Beans"
price_col = "WholesaleUnitPrice"

df_series = df_p[df_p["Commodity"] == commodity][["Date", price_col]].dropna().sort_values("Date")
df_series.set_index("Date", inplace=True)

# Add rolling features
df_series["rolling_mean_7"] = df_series[price_col].rolling(7).mean()
df_series["rolling_std_7"] = df_series[price_col].rolling(7).std()
df_series = df_series.dropna()  # Drop first 7 rows


In [64]:
#Scale features
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df_series)

# Convert back to DataFrame
scaled_df = pd.DataFrame(scaled, index=df_series.index, columns=df_series.columns)


### Creation of sequence (30 days input -> 7 days forecast)


In [65]:
def create_sequences_multi(data, window=30, horizon=7):
    X, y = [], []
    for i in range(window, len(data) - horizon):
        X.append(data[i-window:i])
        y.append(data[i:i+horizon, 0])  # First column is price
    return np.array(X), np.array(y)

window_size = 30
horizon = 7
X, y = create_sequences_multi(scaled_df.values, window_size, horizon)


### Train/test split (last 60 days reserved for testing)


In [66]:
split = len(X) - 60
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


#### LSTM Model 


In [67]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(window_size, X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(horizon))  # Predict next 7 prices

model.compile(optimizer='adam', loss='mse')

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2,
          callbacks=[early_stop], verbose=1)

c:\Users\User\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
860/860 ━━━━━━━━━━━━━━━━━━━━ 35s 33ms/step - loss: 0.0357 - val_loss: 0.0222
Epoch 2/50
860/860 ━━━━━━━━━━━━━━━━━━━━ 26s 31ms/step - loss: 0.0221 - val_loss: 0.0226
Epoch 3/50
860/860 ━━━━━━━━━━━━━━━━━━━━ 27s 32ms/step - loss: 0.0213 - val_loss: 0.0218
Epoch 4/50
860/860 ━━━━━━━━━━━━━━━━━━━━ 26s 31ms/step - loss: 0.0211 - val_loss: 0.0226
Epoch 5/50
860/860 ━━━━━━━━━━━━━━━━━━━━ 26s 30ms/step - loss: 0.0207 - val_loss: 0.0221
Epoch 6/50
860/860 ━━━━━━━━━━━━━━━━━━━━ 27s 31ms/step - loss: 0.0208 - val_loss: 0.0218
Epoch 7/50
860/860 ━━━━━━━━━━━━━━━━━━━━ 26s 30ms/step - loss: 0.0205 - val_loss: 0.0219
Epoch 8/50
860/860 ━━━━━━━━━━━━━━━━━━━━ 26s 30ms/step - loss: 0.0201 - val_loss: 0.0218
Epoch 9/50
860/860 ━━━━━━━━━━━━━━━━━━━━ 26s 30ms/step - loss: 0.0202 - val_loss: 0.0221
Epoch 10/50
860/860 ━━━━━━━━━━━━━━━━━━━━ 40s 28ms/step - loss: 0.0201 - val_loss: 0.0218
Epoch 11/50
860/860 ━━━━━━━━━━━━━━━━━━━━ 25s 29ms/step - loss: 0.0202 - val_loss: 0.0222


### Predict


In [68]:
pred_scaled = model.predict(X_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 468ms/step


In [69]:
# Inverse transform predictions and actuals (only price column)
pred_full = np.zeros((pred_scaled.shape[0], scaled_df.shape[1]))
actual_full = np.zeros_like(pred_full)

pred_full[:, 0] = pred_scaled[:, -1]  # Compare last predicted day
actual_full[:, 0] = y_test[:, -1]     # Compare last actual day

pred_prices = scaler.inverse_transform(pred_full)[:, 0]
actual_prices = scaler.inverse_transform(actual_full)[:, 0]


#### Evaluate


In [70]:
mae = mean_absolute_error(actual_prices, pred_prices)
rmse = mean_squared_error(actual_prices, pred_prices, squared=False)
r2 = r2_score(actual_prices, pred_prices)
mape = np.mean(np.abs((actual_prices - pred_prices) / actual_prices)) * 100

print(f"Tuned LSTM Results for {commodity} ({price_col}):")
print(f"MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.4f}, MAPE: {mape:.2f}%")


Tuned LSTM Results for Beans (WholesaleUnitPrice):
MAE: 23.73, RMSE: 28.18, R²: -0.0062, MAPE: 20.44%


What this does differently:

- Uses all commodities together (more data = better learning).

- Feeds lag, rolling, and seasonal features like we used for XGBoost.

- Adds a Conv1D front-end (captures local shocks) + stacked LSTM for long trends.

- Predicts 7 days ahead, not just 1.

### Prepare Features for All Commodities


In [71]:
df_mc = df_p.copy().dropna(subset=["WholesaleUnitPrice"])  # Focus on wholesale for now
df_mc = df_mc.sort_values("Date")

# Lag features
df_mc["Wholesale_t-1"] = df_mc.groupby("Commodity")["WholesaleUnitPrice"].shift(1)
df_mc["Wholesale_t-7"] = df_mc.groupby("Commodity")["WholesaleUnitPrice"].shift(7)


In [72]:
# Rolling stats
df_mc["RollingMean_7"] = df_mc.groupby("Commodity")["WholesaleUnitPrice"].transform(lambda x: x.rolling(7).mean())
df_mc["RollingMean_14"] = df_mc.groupby("Commodity")["WholesaleUnitPrice"].transform(lambda x: x.rolling(14).mean())


In [73]:
#Seasonality (encode day of week as sin/cos for cyclicity)
df_mc["Date"] = pd.to_datetime(df_mc["Date"], errors="coerce")
df_mc["dayofweek"] = df_mc["Date"].dt.dayofweek
df_mc["dow_sin"] = np.sin(2 * np.pi * df_mc["dayofweek"] / 7)
df_mc["dow_cos"] = np.cos(2 * np.pi * df_mc["dayofweek"] / 7)

# Dropping NA rows (from lag/rolling)
df_mc = df_mc.dropna().reset_index(drop=True)


In [74]:


# Selecting final features
features = ["WholesaleUnitPrice", "Wholesale_t-1", "Wholesale_t-7",
            "RollingMean_7", "RollingMean_14", "dow_sin", "dow_cos"]
target_col = "WholesaleUnitPrice"

# Scale per commodity (fit per commodity for fairness)
scaled_data = []
scalers = {}

for commodity, group in df_mc.groupby("Commodity"):
    scaler = MinMaxScaler()
    scaled_group = scaler.fit_transform(group[features])
    scaled_data.append(pd.DataFrame(scaled_group, columns=features, index=group.index))
    scalers[commodity] = scaler

scaled_df = pd.concat(scaled_data).sort_index()
df_mc[features] = scaled_df


### Build sequences (30-day input -> 7-day forecast) -----


In [75]:
def create_sequences_multivariate(data, window=30, horizon=7, target_idx=0):
    X, y = [], []
    for i in range(window, len(data) - horizon):
        X.append(data[i-window:i])
        y.append(data[i:i+horizon, target_idx])  # Target is first feature (price)
    return np.array(X), np.array(y)

# Build per commodity sequences and combine
all_X, all_y = [], []
for commodity in df_mc["Commodity"].unique():
    group = df_mc[df_mc["Commodity"] == commodity][features].values
    # Skip commodities with too few data points
    if len(group) < 37:  # 30-day window + 7-day horizon
        continue
    
    X_c, y_c = create_sequences_multivariate(group)
    # Only keep non-empty results
    if len(X_c) > 0:
        all_X.append(X_c)
        all_y.append(y_c)

# Concatenate all valid sequences
X = np.concatenate(all_X)
y = np.concatenate(all_y)


In [76]:
# Train/test split (last 10% as test)
split = int(len(X) * 0.9)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


In [77]:
#  Conv1D + LSTM Model 
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(horizon))  # 7-day forecast

model.compile(optimizer='adam', loss='mse')

early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2,
          callbacks=[early_stop], verbose=1)


c:\Users\User\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 38s 23ms/step - loss: 0.0259 - val_loss: 0.0133
Epoch 2/50
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 29s 21ms/step - loss: 0.0183 - val_loss: 0.0141
Epoch 3/50
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 28s 21ms/step - loss: 0.0179 - val_loss: 0.0133
Epoch 4/50
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 29s 21ms/step - loss: 0.0177 - val_loss: 0.0129
Epoch 5/50
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 28s 21ms/step - loss: 0.0176 - val_loss: 0.0131
Epoch 6/50
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 29s 21ms/step - loss: 0.0175 - val_loss: 0.0131
Epoch 7/50
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 28s 21ms/step - loss: 0.0175 - val_loss: 0.0132
Epoch 8/50
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 28s 21ms/step - loss: 0.0173 - val_loss: 0.0130
Epoch 9/50
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 35s 26ms/step - loss: 0.0173 - val_loss: 0.0130
Epoch 10/50
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 40s 30ms/step - loss: 0.0173 - val_loss: 0.0130
Epoch 11/50
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 43s 32ms/step - loss: 0.0173 - val_loss: 0.0129
Epoch 12

In [78]:
# Predict 
pred_scaled = model.predict(X_test)

# Only evaluate last predicted day for simplicity (can extend)
pred_last_day = pred_scaled[:, -1]
actual_last_day = y_test[:, -1]


188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step


In [79]:
# Inverse scaling for evaluation (average scaler since commodities differ)
# Simplify by rescaling using global min/max of all commodities (approximation)
global_scaler = MinMaxScaler()
global_scaler.fit(df_p[["WholesaleUnitPrice"]])  # Fit on raw prices
pred_prices = global_scaler.inverse_transform(pred_last_day.reshape(-1, 1)).flatten()
actual_prices = global_scaler.inverse_transform(actual_last_day.reshape(-1, 1)).flatten()

# Metrics
mae = mean_absolute_error(actual_prices, pred_prices)
rmse = mean_squared_error(actual_prices, pred_prices, squared=False)
r2 = r2_score(actual_prices, pred_prices)
mape = np.mean(np.abs((actual_prices - pred_prices) / actual_prices)) * 100

print(f"Multi-Commodity Conv1D+LSTM Results (Wholesale Prices):")
print(f"MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.4f}, MAPE: {mape:.2f}%")

Multi-Commodity Conv1D+LSTM Results (Wholesale Prices):
MAE: 14.82, RMSE: 22.20, R²: 0.5850, MAPE: 370.87%


The LSTM is still struggling despite the Conv1D + seasonal features:

R² = 0.5882 → better than single-series LSTM (which was near 0) but still far below XGBoost (0.91+).

MAE and RMSE are acceptable, but the MAPE (356%) is absurdly high because many wholesale prices are close to zero (even 0.5), so percentage errors explode.

### CONCLUSION
XGBoost predicts the base price trend (it’s already very strong: R² ≈ 0.92).